# 01. LangGraph 基础概念和节点边

## 课程目标
- 理解 LangGraph 的核心概念
- 学习如何定义节点（Nodes）
- 掌握边（Edges）的创建和连接
- 构建并执行第一个图结构

## 什么是 LangGraph？

LangGraph 是一个用于构建有状态的、多角色的 LLM 应用程序的库。它提供了以下核心功能：

1. **图结构**：将复杂的 LLM 工作流表示为图
2. **状态管理**：在节点之间传递和更新状态
3. **条件逻辑**：基于状态决定执行路径
4. **持久化**：保存和恢复执行状态
5. **人机交互**：支持人工干预和审批

## 1. 环境准备

In [ ]:
# 安装必要的包
!pip install -q langgraph langchain langchain-openai python-dotenv

In [ ]:
# 导入必要的库
from typing import TypedDict, Literal
from langgraph.graph import Graph, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display
import json

## 2. 核心概念：节点（Nodes）

节点是图中的基本执行单元。每个节点：
- 接收状态作为输入
- 执行某些操作
- 返回更新后的状态

让我们创建一些简单的节点：

In [ ]:
# 定义一个简单的状态结构
class State(TypedDict):
    # 消息列表
    messages: list[str]
    # 计数器
    counter: int
    # 处理结果
    result: str

In [ ]:
# 定义节点函数
def node_a(state: State) -> State:
    """第一个节点：初始化消息"""
    print("执行节点 A")
    return {
        "messages": state.get("messages", []) + ["节点A已处理"],
        "counter": state.get("counter", 0) + 1,
        "result": "A处理中"
    }

def node_b(state: State) -> State:
    """第二个节点：处理消息"""
    print("执行节点 B")
    return {
        "messages": state.get("messages", []) + ["节点B已处理"],
        "counter": state.get("counter", 0) + 1,
        "result": "B处理中"
    }

def node_c(state: State) -> State:
    """第三个节点：完成处理"""
    print("执行节点 C")
    return {
        "messages": state.get("messages", []) + ["节点C已完成"],
        "counter": state.get("counter", 0) + 1,
        "result": "处理完成"
    }

## 3. 核心概念：边（Edges）

边定义了节点之间的连接关系。有两种类型的边：
1. **普通边**：无条件地从一个节点流向另一个节点
2. **条件边**：基于某些条件决定流向（后续课程详细介绍）

In [ ]:
# 创建一个简单的线性图
def create_linear_graph():
    # 初始化图
    graph = Graph()
    
    # 添加节点
    graph.add_node("node_a", node_a)
    graph.add_node("node_b", node_b)
    graph.add_node("node_c", node_c)
    
    # 添加边（定义执行顺序）
    graph.add_edge("node_a", "node_b")  # A -> B
    graph.add_edge("node_b", "node_c")  # B -> C
    graph.add_edge("node_c", END)       # C -> 结束
    
    # 设置入口点
    graph.set_entry_point("node_a")
    
    return graph

# 创建图
linear_graph = create_linear_graph()
print("线性图创建成功！")

## 4. 编译和执行图

In [ ]:
# 编译图
app = linear_graph.compile()

# 可视化图结构
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示图形，请确保安装了 graphviz: {e}")
    # 打印图的文本表示
    print(app.get_graph().draw_mermaid())

In [ ]:
# 执行图
initial_state = {
    "messages": ["开始处理"],
    "counter": 0,
    "result": ""
}

print("开始执行图...\n")
result = app.invoke(initial_state)

print("\n执行结果：")
print(json.dumps(result, ensure_ascii=False, indent=2))

## 5. 创建分支图结构

让我们创建一个更复杂的图，包含分支结构：

In [ ]:
# 定义带有分支的节点
def start_node(state: State) -> State:
    """起始节点"""
    print("🚀 开始处理")
    return {
        "messages": ["流程开始"],
        "counter": 1,
        "result": "初始化"
    }

def process_path_1(state: State) -> State:
    """处理路径1"""
    print("📝 执行路径1")
    return {
        "messages": state["messages"] + ["路径1处理"],
        "counter": state["counter"] + 1,
        "result": "路径1完成"
    }

def process_path_2(state: State) -> State:
    """处理路径2"""
    print("📊 执行路径2")
    return {
        "messages": state["messages"] + ["路径2处理"],
        "counter": state["counter"] + 1,
        "result": "路径2完成"
    }

def merge_node(state: State) -> State:
    """合并节点"""
    print("🔄 合并结果")
    return {
        "messages": state["messages"] + ["结果已合并"],
        "counter": state["counter"] + 1,
        "result": "全部完成"
    }

def decide_path(state: State) -> Literal["path1", "path2"]:
    """决策函数：根据状态决定走哪条路径"""
    # 这里可以根据实际逻辑来决定
    # 示例：根据计数器的奇偶性决定
    if state["counter"] % 2 == 0:
        print("➡️ 选择路径2")
        return "path2"
    else:
        print("➡️ 选择路径1")
        return "path1"

In [ ]:
# 创建分支图
def create_branching_graph():
    graph = Graph()
    
    # 添加节点
    graph.add_node("start", start_node)
    graph.add_node("path1", process_path_1)
    graph.add_node("path2", process_path_2)
    graph.add_node("merge", merge_node)
    
    # 设置入口
    graph.set_entry_point("start")
    
    # 添加条件边
    graph.add_conditional_edges(
        "start",
        decide_path,
        {
            "path1": "path1",
            "path2": "path2"
        }
    )
    
    # 两条路径都汇聚到merge节点
    graph.add_edge("path1", "merge")
    graph.add_edge("path2", "merge")
    graph.add_edge("merge", END)
    
    return graph

# 创建并编译分支图
branching_graph = create_branching_graph()
branching_app = branching_graph.compile()

print("分支图创建成功！")

In [ ]:
# 可视化分支图
try:
    display(Image(branching_app.get_graph().draw_mermaid_png()))
except:
    print(branching_app.get_graph().draw_mermaid())

In [ ]:
# 执行分支图
print("执行分支图...\n")
result = branching_app.invoke({})

print("\n最终结果：")
print(json.dumps(result, ensure_ascii=False, indent=2))

## 6. 实践：构建一个简单的对话流程

让我们构建一个更实用的例子：一个简单的对话处理流程

In [ ]:
# 定义对话状态
class ConversationState(TypedDict):
    user_input: str
    intent: str
    response: str
    history: list[str]

# 定义处理节点
def understand_intent(state: ConversationState) -> ConversationState:
    """理解用户意图"""
    user_input = state.get("user_input", "")
    
    # 简单的意图识别逻辑
    if "你好" in user_input or "hello" in user_input.lower():
        intent = "greeting"
    elif "帮助" in user_input or "help" in user_input.lower():
        intent = "help"
    elif "再见" in user_input or "bye" in user_input.lower():
        intent = "goodbye"
    else:
        intent = "general"
    
    print(f"🎯 识别意图: {intent}")
    
    return {
        **state,
        "intent": intent,
        "history": state.get("history", []) + [f"用户: {user_input}"]
    }

def generate_response(state: ConversationState) -> ConversationState:
    """生成回复"""
    intent = state.get("intent", "general")
    
    # 根据意图生成回复
    responses = {
        "greeting": "你好！很高兴见到你。有什么我可以帮助你的吗？",
        "help": "我可以帮你处理各种问题。请告诉我你需要什么帮助。",
        "goodbye": "再见！祝你有美好的一天！",
        "general": "我理解了你的请求。让我来帮你处理这个问题。"
    }
    
    response = responses.get(intent, "抱歉，我不太明白你的意思。")
    print(f"💬 生成回复: {response}")
    
    return {
        **state,
        "response": response,
        "history": state.get("history", []) + [f"助手: {response}"]
    }

def log_conversation(state: ConversationState) -> ConversationState:
    """记录对话"""
    print("📝 对话已记录")
    return state

In [ ]:
# 构建对话处理图
def create_conversation_graph():
    graph = Graph()
    
    # 添加节点
    graph.add_node("understand", understand_intent)
    graph.add_node("respond", generate_response)
    graph.add_node("log", log_conversation)
    
    # 设置流程
    graph.set_entry_point("understand")
    graph.add_edge("understand", "respond")
    graph.add_edge("respond", "log")
    graph.add_edge("log", END)
    
    return graph

# 编译对话图
conversation_graph = create_conversation_graph()
conversation_app = conversation_graph.compile()

print("对话处理图创建成功！")

In [ ]:
# 测试对话系统
test_inputs = [
    "你好",
    "我需要帮助",
    "今天天气怎么样",
    "再见"
]

for user_input in test_inputs:
    print(f"\n{'='*50}")
    print(f"用户输入: {user_input}")
    print(f"{'='*50}")
    
    result = conversation_app.invoke({
        "user_input": user_input,
        "history": []
    })
    
    print(f"\n最终回复: {result['response']}")
    print(f"对话历史: {result['history']}")

## 7. 流式执行

LangGraph 支持流式执行，可以实时观察图的执行过程：

In [ ]:
# 流式执行示例
print("流式执行对话处理...\n")

for event in conversation_app.stream({
    "user_input": "你好，请帮助我",
    "history": []
}):
    # 打印每个节点的执行结果
    for node_name, node_output in event.items():
        print(f"\n节点 [{node_name}] 输出:")
        print(json.dumps(node_output, ensure_ascii=False, indent=2))

## 8. 练习题

### 练习 1：创建一个数据处理管道
创建一个图，包含以下节点：
1. 数据验证节点
2. 数据清洗节点
3. 数据转换节点
4. 数据保存节点

In [ ]:
# 在这里实现你的数据处理管道
class DataState(TypedDict):
    raw_data: list
    valid_data: list
    clean_data: list
    transformed_data: list
    status: str

def validate_data(state: DataState) -> DataState:
    """验证数据"""
    # 实现验证逻辑
    pass

def clean_data(state: DataState) -> DataState:
    """清洗数据"""
    # 实现清洗逻辑
    pass

def transform_data(state: DataState) -> DataState:
    """转换数据"""
    # 实现转换逻辑
    pass

def save_data(state: DataState) -> DataState:
    """保存数据"""
    # 实现保存逻辑
    pass

# 构建并测试你的图

### 练习 2：添加错误处理
修改上面的对话处理图，添加错误处理节点，当意图识别失败时，走错误处理路径。

In [ ]:
# 在这里实现带错误处理的对话图


## 总结

在本课中，我们学习了：

1. **LangGraph 的核心概念**
   - 图、节点、边的基本概念
   - 状态在节点间的传递

2. **节点的定义和使用**
   - 节点函数的编写
   - 状态的接收和返回

3. **边的创建和连接**
   - 普通边的使用
   - 条件边的基础介绍

4. **图的构建和执行**
   - 图的编译
   - 同步和流式执行

## 下一课预告

在下一课中，我们将深入学习：
- StateGraph 的详细使用
- 状态管理的高级特性
- Reducer 函数的应用
- 状态注解和类型系统